### 1.Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

In [5]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.preprocessing import StandardScaler

import time
import datetime
import matplotlib.pyplot as plt


features = pd.read_csv('./features.csv', index_col='match_id')


y = features['radiant_win'].values

features = features.drop(['duration', 'radiant_win', 'tower_status_radiant', 'tower_status_dire', \
                  'barracks_status_radiant', 'barracks_status_dire'], axis = 1)


features = features.fillna(0)


sdvig = KFold(n_splits = 5, shuffle = True)


def regressia(X, y):
    if isinstance(X, pd.DataFrame):
        X = X.values
    maxxx = -1
    best = -1

    for C in list(np.power(10.0, np.arange(-2, 2))):
        print("C =", C)
    
        nachalo = datetime.datetime.now()
        clf = LogisticRegression(C = C)
    
        scores = np.array([])
        for train_index, test_index in sdvig.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            
            scaler = StandardScaler()
            scaler.fit(X_train, y_train)
            scaled_train_X = scaler.transform(X_train)
            scaled_test_X = scaler.transform(X_test)
        
            clf  = clf.fit(scaled_train_X, y_train)
            pred = clf.predict_proba(scaled_test_X)[:, 1]
            scores = np.append(scores, roc_auc_score(y_test, pred))
        
        print(np.median(scores))
    
        current_max = max(scores)
        if current_max > maxxx:
            maxxx = current_max
            best = C
    
        print('Proshlo vremya:', datetime.datetime.now() - nachalo)
    
    return maxxx, best


score, C = regressia(features, y)

print('Maximalnaya mediana:', score)
print('Luchshee C:', C)


C = 0.01
0.7171582328987263
Proshlo vremya: 0:01:29.180629
C = 0.1
0.7152693073725409
Proshlo vremya: 0:01:28.218257
C = 1.0
0.7159970266280601
Proshlo vremya: 0:01:37.500451
C = 10.0
0.7163091612209762
Proshlo vremya: 0:01:24.795463
Maximalnaya mediana: 0.721545908220063
Luchshee C: 10.0


### Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?

In [6]:
score, C = regressia(features.drop(['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', \
                          'd3_hero', 'd4_hero', 'd5_hero'], axis=1), y)

print('Maximalnaya mediana:', score)
print('Luchshee C:', C)

C = 0.01
0.7172949256863698
Proshlo vremya: 0:01:28.178036
C = 0.1
0.7168958536124972
Proshlo vremya: 0:01:26.891839
C = 1.0
0.7162833028952204
Proshlo vremya: 0:01:30.000073
C = 10.0
0.7162729038477981
Proshlo vremya: 0:01:31.599760
Maximalnaya mediana: 0.7219516244111166
Luchshee C: 1.0


### На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).

In [7]:
unikal_geroi = len(np.unique(features[['r' + str(i) + '_hero' for i in range(1, 6)]]))
unikal_geroi

108

### Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.

In [8]:
X_pick = np.zeros((features.shape[0], 112))

for i, match_id in enumerate(features.index):
    for p in range(5):
        X_pick[i, features.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, features.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1


X_pick


obrabotka = features.drop(['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', \
                          'd3_hero', 'd4_hero', 'd5_hero'], axis=1)
X = np.concatenate((obrabotka, X_pick), axis = 1)

### Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?

In [9]:
score, C = regressia(X, y)

print('Maximalnaya mediana:', score)
print('Luchshee C:', C)

C = 0.01
0.7521941688062801
Proshlo vremya: 0:02:17.192596
C = 0.1
0.7533153377072721
Proshlo vremya: 0:02:40.297457
C = 1.0
0.7524957261045382
Proshlo vremya: 0:02:36.599070
C = 10.0
0.7512482986986649
Proshlo vremya: 0:02:37.398586
Maximalnaya mediana: 0.7565595748920827
Luchshee C: 10.0


### Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).

In [2]:
X_test = pd.read_csv('./features_test.csv', index_col='match_id')
X_train = StandardScaler().fit_transform(X)
X_test = X_test.fillna(0)

X_pick = np.zeros((X_test.shape[0], 112))
for i, match_id in enumerate(X_test.index):
    for p in range(5):
        X_pick[i, X_test.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, X_test.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

X_test = X_test.drop(['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', \
                          'd3_hero', 'd4_hero', 'd5_hero'], axis=1)
X_test = np.hstack((X_test.values, X_pick))
X_test = StandardScaler().fit_transform(X_test)

clf = LogisticRegression(C=C)
clf = clf.fit(X_train, y)
pred = clf.predict_proba(X_test)[:, 1]
pred

print('Minimum:', min(pred))
print('Maximum:', max(pred))

Minimum: 0.008721474723126527
Maximum: 0.9963305890721987


## Отчёт

1 Какое качество получилось у логистической регрессии над всеми исходными признаками? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?
Лучшее качество получилось 0.721 при C=0.01, больше чем при градиентном бустинге. Градиентый бустинг хорошо показывает себя на задачах где признаки хорошо сгруппированны в пространстве, т.к. ориентирован на нахождение таких многомерных скоплений объектов. Логистическая регрессия же позволяет найти гиперплоскость максимально разделяющую объекты двух классов - она лучше работает когда объекты слабо сгруппированы в пространстве.

2 Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? Чем вы можете объяснить это изменение?
Качество не изменилось, так как линейная модель плохо работает с категориальными данными, без обработки.

3 Сколько различных идентификаторов героев существует в данной игре?
108 героев

4 Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить?
Качество предсказаний классификатора выросло. При использовании "мешка слов" можно иметь в виду сильные стороны героя.

5 Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?
Минимальное значение - 0.0087, максимальное – 0.9963, среднее арифметическое - 0.5